In [1]:
import pandas as pd
import emsi
import queries
from sqlalchemy import create_engine
from secrets import secrets

### Get access to Emsi

In [2]:
access_token = emsi.get_access_token(secrets['emsi_client_id'], secrets['emsi_secret'])
emsi.check_health(access_token)

{'healthy': True, 'message': 'Service is healthy'}

### Load existing skills

In [3]:
f = open("skills/initial.txt", "r")
skills = []
for x in f:
    skills.append(x[:-1])

len(skills)

196

In [4]:
df_own = pd.DataFrame(columns=['Skill'])
df_own['Skill'] = skills
df_own.head()

,Skill
0,AB Testing
1,AIML
2,API
3,AWS
4,Adaptive Insights


### Query existing skills

In [5]:
training_engine = create_engine(secrets['training_db'])
df_train = pd.read_sql_query(queries.skills_query, con=training_engine)
training_engine.dispose()
df_train.head()

,Name
0,Linear Algebra
1,Basic Calculus
2,DBMS
3,Hadoop
4,R Programming


### Get related skills from Emsi

In [9]:
def get_related_skills(skills, print_progress=True):
    new_skills = []
    for i, s in enumerate(skills):
        if (i + 1) % int(len(skills) / 10) == 0:
            print("{}% done".format((i + 1) / len(skills) * 100))
        related = emsi.get_emsi_skills(s, access_token)
        for r in related:
            new_skills.append(r['name'])
    return new_skills

In [10]:
# From own
related_own = get_related_skills(df_own['Skill'].to_list())
len(related_own)

9.693877551020408% done
19.387755102040817% done
29.081632653061224% done
38.775510204081634% done
48.46938775510204% done
58.16326530612245% done
67.85714285714286% done
77.55102040816327% done
87.24489795918367% done
96.93877551020408% done


783

In [11]:
df1 = pd.DataFrame(columns=['Skill', 'Source'])
df1['Skill'] = related_own
df1['Source'] = 'Own'
df1.head()

,Skill,Source
0,Artificial Intelligence Markup Language (AIML),Own
1,Application Programming Interface (API),Own
2,RESTful API,Own
3,Java Persistence API,Own
4,API Design,Own


In [12]:
# From own
related_training = get_related_skills(df_train['Name'].to_list())
len(related_training)

8.823529411764707% done
17.647058823529413% done
26.47058823529412% done
35.294117647058826% done
44.11764705882353% done
52.94117647058824% done
61.76470588235294% done
70.58823529411765% done
79.41176470588235% done
88.23529411764706% done
97.05882352941177% done


134

In [13]:
df2 = pd.DataFrame(columns=['Skill', 'Source'])
df2['Skill'] = related_training
df2['Source'] = 'Training'
df2.head()

,Skill,Source
0,Linear Algebra,Training
1,Elementary Matrix (Linear Algebra),Training
2,Basic Linear Algebra Subprograms,Training
3,Numerical Linear Algebra,Training
4,Linear Algebra Package (LAPACK),Training


In [14]:
# From own
related_own = get_related_skills(df_own['Skill'].to_list())
len(related_own)

df1 = pd.DataFrame(columns=['Skill', 'Source'])
df1['Skill'] = related_own
df1['Source'] = 'Own'
df1.head()

9.693877551020408% done
19.387755102040817% done
29.081632653061224% done
38.775510204081634% done
48.46938775510204% done
58.16326530612245% done
67.85714285714286% done
77.55102040816327% done
87.24489795918367% done
96.93877551020408% done


,Skill,Source
0,Artificial Intelligence Markup Language (AIML),Own
1,Application Programming Interface (API),Own
2,RESTful API,Own
3,Java Persistence API,Own
4,API Design,Own


### Consolidate

In [21]:
df_emsi = pd.concat([df1, df2])
df_emsi.head()

,Skill,Source
0,Artificial Intelligence Markup Language (AIML),Own
1,Application Programming Interface (API),Own
2,RESTful API,Own
3,Java Persistence API,Own
4,API Design,Own


In [22]:
df_emsi.describe(include='object')

,Skill,Source
count,917,917
unique,727,2
top,Github,Own
freq,4,783


In [27]:
df_emsi = df_emsi.drop_duplicates(subset=['Skill'])
df_emsi = df_emsi.sort_values(by=['Skill'])
df_emsi = df_emsi.reset_index().drop(columns=['index'])
df_emsi.head()

,Skill,Source
0,.NET Framework,Own
1,3D Modeling,Own
2,API Design,Own
3,API Gateway,Own
4,API Management,Own


In [28]:
df_emsi.describe(include='object')

,Skill,Source
count,727,727
unique,727,2
top,Cloudera Impala,Own
freq,1,645


In [29]:
df_emsi.to_csv('skills/test.csv')